In [1]:
#import libraries
import pandas as pd
import re
import requests
import time
from io import StringIO
import os
import json

In [2]:
# get rid of parentheses 
# white space before

In [ ]:
#config 
API_KEY = "tSUXiFeF9OmTfzi3x24XsaKvGavaNFUMzQHUaABw"
PROGRESS_FILE = "/Users/sanyakhan/Downloads/parsing_progress_tester1009.json"
OUTPUT_FILE = "/Users/sanyakhan/Downloads/congressional_speeches_parsed_tester1009.csv"
file_path = '/Users/sanyakhan/Downloads/newtest - Sheet1.csv'
df = pd.read_csv(file_path, skiprows=3)
df

,Title,URL,Issue Date,Section,Volume,Number,Page
0,REINTRODUCING THE SERVICES FOR ENDING LONG-TER...,https://www.congress.gov/congressional-record/...,06/13/2019,NaN,165,99,E770
1,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,10/15/2019,NaN,165,162,H8135
2,HOMELESS ASSISTANCE ACT OF 2019; Congressional...,https://www.congress.gov/congressional-record/...,01/13/2020,NaN,166,7,H186
3,HOMELESS ASSISTANCE ACT OF 2019; Congressional...,https://www.congress.gov/congressional-record/...,01/14/2020,NaN,166,8,E37
4,TRIBAL ACCESS TO HOMELESS ASSISTANCE ACT; Cong...,https://www.congress.gov/congressional-record/...,11/18/2019,NaN,165,184,H8926
5,LEGAL SERVICES FOR HOMELESS VETERANS ACT; Cong...,https://www.congress.gov/congressional-record/...,02/25/2020,NaN,166,37,H1176
6,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,10/15/2019,NaN,165,162,H8138
7,HOUSING UNHOUSED DISABLED VETERANS ACT; Congre...,https://www.congress.gov/congressional-record/...,02/10/2025,NaN,171,27,H603
8,HOMELESS ASSISTANCE ACT OF 2019; Congressional...,https://www.congress.gov/congressional-record/...,01/13/2020,NaN,166,7,H186
9,McKINNEY-VENTO HOMELESS ASSISTANCE ACT; Congre...,https://www.congress.gov/congressional-record/...,07/20/2017,NaN,163,123,S4108


In [4]:
# just loading
file_path = file_path = '/Users/sanyakhan/Downloads/search_results_2025-10-14_0302am.csv'
df = pd.read_csv(file_path, skiprows=2)

# so check if the progress file exists
def load_progress():
    if os.path.exists(PROGRESS_FILE):
        try:
            with open(PROGRESS_FILE, 'r') as f:
                progress = json.load(f)
            print(f"Loaded progress: {progress['processed_rows']} rows already processed")
            return progress
        except:
            print("Could not load progress file, starting fresh")
    return {'processed_rows': 0, 'processed_urls': set()}

def save_progress(processed_rows, processed_urls):
    progress = {
        'processed_rows': processed_rows,
        'processed_urls': list(processed_urls)
    }
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress, f)



def append_to_csv(speeches, file_path):
   #if speeches list empty then return
    if not speeches:
        return
    
    #convert the dicionary to dataframe
    speeches_df = pd.DataFrame(speeches)
    if 'Party' not in speeches_df.columns:
        speeches_df['Party'] = pd.NA
    if 'State' not in speeches_df.columns:
        speeches_df['State'] = pd.NA
    # reorder columns
    column_order = [ 'Chamber', 'Session', 'Year', 'Date', 'Speaker', 'Party', 'State' , 'Bill Reference', 'Title', 'URL', 'Volume', 'Number', 'Section', 'turn_number', 'speaker_title', 'speech_text', 'speech_length_words', 'granule_id', 'package_id' ]
    # give the dataframe the columns
    speeches_df = speeches_df[[col for col in column_order if col in speeches_df.columns]]
    
    # append or create file to put speeches in
    if os.path.exists(file_path):
        speeches_df.to_csv(file_path, mode='a', header=False, index=False)
    else:
        speeches_df.to_csv(file_path, mode='w', header=True, index=False)

# uses the url to get where the speech is from (chamber)
def get_chamber(url):
    if pd.isna(url) or not isinstance(url, str):
        return None
    url = url.lower()
    if "senate-section" in url:
        return "Senate"
    elif "house-section" in url:
        return "House"
    elif "extensions-of-remarks" in url:
        return "Extensions"
    return None

# looking in title column of the csv
def get_bill_reference(title):
    if pd.isna(title) or not isinstance(title, str):
        return None
    pattern = r'\b(H\.R\.|S\.|H\.Res\.|S\.Res\.|H\.J\.Res\.|S\.J\.Res\.)\s*\d+\b'
    matches = [m.group(0) for m in re.finditer(pattern, title)]
    return ', '.join(sorted(set(matches))) if matches else None

# parses the date into something govinfo can use
def construct_crec_package_id(date_str):
    if pd.isna(date_str):
        return None
    try:
        date_obj = pd.to_datetime(date_str)
        return f"CREC-{date_obj.strftime('%Y-%m-%d')}"
    except:
        return None

# building URL
#this is essneitally making sure the url is valid
def get_package_granules(package_id, api_key):
    url = f"https://api.govinfo.gov/packages/{package_id}/granules"
    params = {'api_key': api_key, 'offsetMark': '*', 'pageSize': 1000}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Package not found (404)")
        else:
            print(f"HTTP Error {e.response.status_code}")
        return None
    except Exception as e:
        print(f"Error getting granules: {e}")
        return None

# get meta data 
def get_granule_content(package_id, granule_id, api_key):
    url = f"https://api.govinfo.gov/packages/{package_id}/granules/{granule_id}/summary"
    params = {'api_key': api_key}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error getting granule details: {e}")
        return None
    
# gets raw speech data
def get_granule_text(package_id, granule_id, api_key):
    url = f"https://api.govinfo.gov/packages/{package_id}/granules/{granule_id}/htm"
    params = {'api_key': api_key}
    
    try:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f" Error downloading text: {e}")
        return None


#now once we have the speech, we have all the text, we need to parse it by speaker
def parse_speeches_from_text(text):
    if not text:
        return []
    speeches = []
    #regex pattern to identify speaker introduction

    speaker_pattern = r'\n\s*((?:Mr\.|Ms\.|Mrs\.|Senator|The)\s+[A-Z][A-Z\s]+?)(?:\s+of\s+[A-Z][a-z]+)?\.\s'
    
    # Find all speaker introductions
    matches = list(re.finditer(speaker_pattern, text, re.MULTILINE))
    if not matches:
        return []
    
    # Extract speeches between speaker introductions
    for i, match in enumerate(matches):
        speaker_raw = match.group(1).strip()
        start_pos = match.end()
        

        # end position is either the next speaker or end of text
        end_pos = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        
        speech_text = text[start_pos:end_pos].strip()
        
        # dealing w/ whitesapce
        speech_text = re.sub(r'\s+', ' ', speech_text) 
        
        # get speaker info
        title_match = re.match(r'(Mr\.|Ms\.|Mrs\.|Senator|The)\s+(.+)', speaker_raw)
        if title_match:
            title = title_match.group(1)
            name = title_match.group(2).strip()
        else:
            title = None
            name = speaker_raw
        
        # hmmm interesting idk if this is needed
        if len(speech_text) < 20:
            continue
        
        #adding to speech list 
        speeches.append({
            'speaker_title': title,
            'speaker_name': name.title(),  # Convert from ALL CAPS
            'speech_text': speech_text,
            'speech_length_words': len(speech_text.split()),
            'turn_number': len(speeches) + 1
        })
    
    return speeches


# ok now we are actually pulling from the url
def process_congressional_record(row, api_key):
    url = row.get("URL")
    date = row.get("Date")
    title = row.get("Title")
    
    if pd.isna(url) or pd.isna(date):
        return []
    
    print(f"🔍 Processing: {url}")
    
    # Construct package ID
    package_id = construct_crec_package_id(date)
    if not package_id:
        print("Could not construct package ID")
        return []
    
    print(f" Package ID: {package_id}")
    
    # Get granules
    granules_data = get_package_granules(package_id, api_key)
    if not granules_data or 'granules' not in granules_data:
        print("No granules found")
        return []
    
    granules = granules_data.get('granules', [])
    print(f"Found {len(granules)} granules")
    
    # Extract section from URL
    section_match = re.search(r'article/([A-Z])(\d+)(?:-\d+)?', url)
    if not section_match:
        print("Could not extract section")
        return []
    
    section_letter = section_match.group(1)
    section_number = section_match.group(2)
    section_pattern = f"Pg{section_letter}{section_number}"
    
    print(f"Looking for pattern: {section_pattern}")
    
    # Find matching granule
    matching_granule = None
    for granule in granules:
        granule_id = granule.get('granuleId', '')
        if section_pattern in granule_id:
            matching_granule = granule
            print(f"Found matching granule: {granule_id}")
            break
    
    if not matching_granule:
        print(f"No matching granule found")
        return []
    
    granule_id = matching_granule['granuleId']
    
    # Get granule metadata
    granule_detail = get_granule_content(package_id, granule_id, api_key)

    # Get full text
    print(f"Downloading text...")
    text = get_granule_text(package_id, granule_id, api_key)
    
    if not text:
        print(f"Could not download text")
        return []
    
    # Parse speeches
    print(f"Parsing speeches...")
    speeches = parse_speeches_from_text(text)
    
    if not speeches:
        print(f"No speeches found in text")
        return []
    
    print(f"Found {len(speeches)} speeches")
    
    # Add metadata to each speech
    speech_records = []
    for speech in speeches:
        record = {
            'Chamber': get_chamber(url),
            'Session': row.get('Volume'),
            'Year': row.get('Year'),
            'Date': date,
            'Speaker': speech['speaker_name'],
            'Bill Reference': get_bill_reference(title),
            'Title': title,
            'URL': url,
            'Volume': row.get('Volume'),
            'Number': row.get('Number'),
            'Section': row.get('Section'),
            'speaker_title': speech['speaker_title'],
            'speech_text': speech['speech_text'],
            'speech_length_words': speech['speech_length_words'],
            'turn_number': speech['turn_number'],
            'granule_id': granule_id,
            'package_id': package_id
        }
        speech_records.append(record)
    
    return speech_records

# Parse dates
issue_date = pd.to_datetime(df["Issue Date"], errors="coerce")
df["Date"] = issue_date.dt.date
df["Year"] = issue_date.dt.year

# Process records
print("\n" + "=" * 60)
print("STARTING CONGRESSIONAL RECORD SPEECH PARSING")
print("=" * 60 + "\n")

# Load previous progress
progress = load_progress()
processed_urls = set(progress.get('processed_urls', []))
start_row = progress.get('processed_rows', 0)

# Limit processing for testing
MAX_ROWS_TO_PROCESS = None  # Set to None to process ALL rows
rows_to_process = df.head(MAX_ROWS_TO_PROCESS) if MAX_ROWS_TO_PROCESS else df

# Skip already processed rows
if start_row > 0:
    rows_to_process = rows_to_process.iloc[start_row:]
    print(f" Resuming from row {start_row + 1}")

total_speeches_saved = 0
success_count = 0
failure_count = 0
batch_speeches = []  # Collect speeches for batch saving
SAVE_EVERY = 10  # Save progress every N rows

print(f"Total rows to process: {len(rows_to_process)}")
print(f"Progress will be saved every {SAVE_EVERY} rows\n")

for i, (idx, row) in enumerate(rows_to_process.iterrows(), 1):
    actual_row_num = start_row + i
    print(f"\n[{actual_row_num}/{len(df)}]", end=" ")
    
    url = row.get('URL')
    
    # Skip if already processed, json
    if url in processed_urls:
        print(f" Skipping (already processed)")
        continue
    
    # deal w /rate limiting
    time.sleep(0.5)
    
    # Process record
    speeches = process_congressional_record(row, API_KEY)
    
    if speeches:
        batch_speeches.extend(speeches)
        success_count += 1
        total_speeches_saved += len(speeches)
    else:
        failure_count += 1
    
    #mark as processed
    processed_urls.add(url)
    
    # save progress periodically
    if i % SAVE_EVERY == 0:
        print(f"\n Saving progress...")
        append_to_csv(batch_speeches, OUTPUT_FILE)
        save_progress(actual_row_num, processed_urls)
        batch_speeches = []  # Clear batch
        
        print(f"📊 Progress Update:")
        print(f"   Processed: {actual_row_num}/{len(df)} rows")
        print(f"   Successes: {success_count} | Failures: {failure_count}")
        print(f"   Total speeches saved: {total_speeches_saved}")
        
#save any remaining speeches
if batch_speeches:
    print(f"\n Saving final batch...")
    append_to_csv(batch_speeches, OUTPUT_FILE)
    total_speeches_saved += len(batch_speeches)

# Save final progress
save_progress(len(df), processed_urls)

# idg what thsi si 
print("\n" + "=" * 60)
print("PROCESSING COMPLETE")
print("=" * 60)

if os.path.exists(OUTPUT_FILE):
    # Load final results
    final_df = pd.read_csv(OUTPUT_FILE)
    
    
    # Show sample
    print("\n" + "=" * 60)
    print("SAMPLE DATA (first 3 speeches)")
    print("=" * 60)
    for idx, speech in final_df.head(3).iterrows():
        print(f"\nSpeech {idx + 1}:")
        print(f"  Title: {speech['Title'][:60]}...")
        print(f"  Turn: {speech['turn_number']}")
        print(f"  Words: {speech['speech_length_words']}")
        print(f"  Text preview: {speech['speech_text'][:150]}...")
    
    print(f"\n Final dataset saved to '{OUTPUT_FILE}'")
    
else:
    print("No speeches were successfully parsed")
    print("Check the error messages above for details")

print("\n" + "=" * 60)


STARTING CONGRESSIONAL RECORD SPEECH PARSING

Loaded progress: 7 rows already processed
 Resuming from row 8
Total rows to process: 0
Progress will be saved every 10 rows


PROCESSING COMPLETE

SAMPLE DATA (first 3 speeches)

Speech 1:
  Title: REINTRODUCING THE SERVICES FOR ENDING LONG-TERM HOMELESSNESS...
  Turn: 1
  Words: 329
  Text preview: Madam Speaker, I rise today to congratulate Norwich Superintendent of Public Schools, Abby Dolliver, on a career in education spanning 33 years, inclu...

Speech 2:
  Title: HOMELESS VETERAN FAMILIES ACT; Congressional Record Vol. 165...
  Turn: 1
  Words: 234
  Text preview: Mr. Speaker, I move to suspend the rules and pass the bill (H.R. 95) to amend title 38, United States Code, to ensure that children of homeless vetera...

Speech 3:
  Title: HOMELESS VETERAN FAMILIES ACT; Congressional Record Vol. 165...
  Turn: 2
  Words: 49
  Text preview: Mr. Speaker, I ask unanimous consent that all Members may have 5 legislative days in which to revi

In [5]:
out_df = pd.read_csv(OUTPUT_FILE, dtype=str)
out_df.head(5)

,Chamber,Session,Year,Date,Speaker,Party,State,Bill Reference,Title,URL,Volume,Number,Section,turn_number,speaker_title,speech_text,speech_length_words,granule_id,package_id
0,Extensions,165,2019,2019-06-13,Courtney,NaN,NaN,NaN,REINTRODUCING THE SERVICES FOR ENDING LONG-TER...,https://www.congress.gov/congressional-record/...,165,99,NaN,1,Mr.,"Madam Speaker, I rise today to congratulate No...",329,CREC-2019-06-13-pt1-PgE770,CREC-2019-06-13
1,House,165,2019,2019-10-15,Takano,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,1,Mr.,"Mr. Speaker, I move to suspend the rules and p...",234,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
2,House,165,2019,2019-10-15,Takano,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,2,Mr.,"Mr. Speaker, I ask unanimous consent that all ...",49,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
3,House,165,2019,2019-10-15,Takano,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,3,Mr.,"Mr. Speaker, I yield myself such time as I may...",500,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
4,House,165,2019,2019-10-15,David P,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,4,Mr.,"ROE of Tennessee. Mr. Speaker, I yield myself ...",321,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15


In [6]:
members_fp = "/Users/sanyakhan/Downloads/legislators_combined.csv"
members_df = pd.read_csv(members_fp, dtype=str)
members_df.tail(5)

,bioguide,first_name,last_name,official_full,birthday,gender,party,state,type,start,end,url,phone,address,office
12755,H001104,Jon,Husted,Jon Husted,1967-08-25,M,Republican,OH,sen,2025-01-21,2029-01-03,https://www.husted.senate.gov,202-224-3353,304 Russell Senate Office Building Washington ...,304 Russell Senate Office Building
12756,M001244,Ashley,Moody,Ashley Moody,1975-03-28,F,Republican,FL,sen,2025-01-21,2029-01-03,https://www.moody.senate.gov,202-224-3041,387 Russell Senate Office Building Washington ...,387 Russell Senate Office Building
12757,P000622,Jimmy,Patronis,Jimmy Patronis,1972-04-13,M,Republican,FL,rep,2025-04-02,2027-01-03,https://patronis.house.gov,202-225-4136,2021 Rayburn House Office Building Washington ...,2021 Rayburn House Office Building
12758,F000484,Randall,Fine,Randy Fine,1974-04-20,M,Republican,FL,rep,2025-04-02,2027-01-03,https://fine.house.gov,202-225-2706,244 Cannon House Office Building Washington DC...,244 Cannon House Office Building
12759,W000831,James,Walkinshaw,NaN,1982-10-22,M,Democrat,VA,rep,2025-09-10,2027-01-03,NaN,202-225-1492,2265 Rayburn House Office Building Washington ...,2265 Rayburn House Office Building


In [7]:
# just normalizing
def norm_speech(x):
    if not isinstance(x, str): return None
    s = x
    if "house" in s:  return "HOUSE"
    if "senate" in s: return "SENATE"
    return None

def norm_member(x):
    if not isinstance(x, str): return None
    s = x
    if s in {"rep"}: return "HOUSE"
    if s in {"sen"}:  return "SENATE"
    return None



def last_name(name):
    if not isinstance(name, str) or not name.strip():
        return []
    s = name.upper().strip() # all caps
    s = re.sub(r"[^\w\s'\-]", " ", s) #removes periods 
    s = re.sub(r"\s+", " ", s) # rem spaces
    parts = s.split(" ") 
    full_last = " ".join(parts)
    return [full_last] #dealing with multi 


In [8]:
mem_df = pd.read_csv(members_fp, dtype=str)
mem_df.columns = [c.lower() for c in mem_df.columns]
mem_df["last_name_norm"] = mem_df["last_name"].fillna("").str.upper().str.strip()
mem_df["start_dt"] = pd.to_datetime(mem_df["start"], errors="coerce")
mem_df["end_dt"] = pd.to_datetime(mem_df["end"], errors="coerce").fillna(pd.Timestamp("2100-01-01"))


#dealing w chamber 
if "type" in mem_df.columns:
    mem_df["member_type"] = mem_df["type"].map(norm_member)
else:
    mem_df["member_type"] = pd.NA


#look upsssss
by_full_last = {}
for i, r in mem_df.iterrows():
    key = r.get("last_name_norm", "")
    if isinstance(key, str) and key.strip():
        by_full_last.setdefault(key, []).append(i)

In [76]:
# ok now matching function 

def pick_candidate(speaker, speech_date,  speech_type):
    if not isinstance(speaker, str) or not speaker.strip():
        return None, None
    opts = last_name(speaker)
    full_last = opts[0] if opts else None
    if not full_last:
        return None, None
    cand_idx = set(by_full_last.get(full_last, []))
    if not cand_idx:
        return None, None
    cands = mem_df.loc[list(cand_idx)].copy()
    if isinstance(speech_type, str):
        strict = cands[(cands["member_type"].isna()) | (cands["member_type"]== speech_type)]
        if strict.empty:
            return None, None
    else:
        strict = cands
    if len(strict) == 1:
        r = strict.iloc[0]
        return str(r["party"]).strip(), str(r["state"]).strip()
    ps = strict[["party","state"]].dropna()
    if not ps.empty and len(ps.drop_duplicates()) == 1:
        r = ps.iloc[0]
        return str(r["party"]).strip(), str(r["state"]).strip()
    return None, None

out_df["speech_type"]= out_df["Chamber"].map(norm_speech) if "Chamber" in out_df.columns else None
out_df["Date"] = pd.to_datetime(out_df["Date"], errors="coerce")
mask_to_fix = out_df["Party"].isna() | out_df["State"].isna()
filled_party = filled_state = 0


for i in out_df.index[mask_to_fix]:
    party0, state0 = out_df.at[i,"Party"], out_df.at[i,"State"]
    party, state = pick_candidate(out_df.at[i,"Speaker"], out_df.at[i,"Date"], out_df.at[i,"speech_type"])
    if pd.isna(party0) and party:
        out_df.at[i,"Party"] = party
        filled_party += 1
    if pd.isna(state0) and state:
        out_df.at[i,"State"] = state
        filled_state += 1

out_df.drop(columns=["speech_type"], errors="ignore", inplace=True)
print(f"Filled Party: {filled_party} | Filled State: {filled_state}")

out_df

Filled Party: 0 | Filled State: 0


,Chamber,Session,Year,Date,Speaker,Party,State,Bill Reference,Title,URL,Volume,Number,Section,turn_number,speaker_title,speech_text,speech_length_words,granule_id,package_id
0,Extensions,165,2019,2019-06-13,Courtney,NaN,NaN,NaN,REINTRODUCING THE SERVICES FOR ENDING LONG-TER...,https://www.congress.gov/congressional-record/...,165,99,NaN,1,Mr.,"Madam Speaker, I rise today to congratulate No...",329,CREC-2019-06-13-pt1-PgE770,CREC-2019-06-13
1,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,1,Mr.,"Mr. Speaker, I move to suspend the rules and p...",234,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
2,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,2,Mr.,"Mr. Speaker, I ask unanimous consent that all ...",49,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
3,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,3,Mr.,"Mr. Speaker, I yield myself such time as I may...",500,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
4,House,165,2019,2019-10-15,David P,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,4,Mr.,"ROE of Tennessee. Mr. Speaker, I yield myself ...",321,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
5,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,5,Mr.,"Mr. Speaker, I yield 3 minutes to the gentlewo...",28,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
6,House,165,2019,2019-10-15,Brownley,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,6,Ms.,"Mr. Speaker, I thank the chairman for yielding...",435,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
7,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,7,Mr.,I yield the gentlewoman from California an add...,10,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
8,House,165,2019,2019-10-15,Brownley,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,8,Ms.,"Mr. Speaker, providing housing to our Nation's...",116,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
9,House,165,2019,2019-10-15,David P,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,9,Mr.,"ROE of Tennessee. Mr. Speaker, I yield 5 minut...",564,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15


In [42]:
members_df

,bioguide,first_name,last_name,official_full,birthday,gender,party,state,type,start,end,url,phone,address,office
0,B000226,Richard,Bassett,NaN,1745-04-02,M,Anti-Administration,DE,sen,1789-03-04,1793-03-03,NaN,NaN,NaN,NaN
1,B000546,Theodorick,Bland,NaN,1742-03-21,M,NaN,VA,rep,1789-03-04,1791-03-03,NaN,NaN,NaN,NaN
2,B001086,Aedanus,Burke,NaN,1743-06-16,M,NaN,SC,rep,1789-03-04,1791-03-03,NaN,NaN,NaN,NaN
3,C000187,Daniel,Carroll,NaN,1730-07-22,M,NaN,MD,rep,1789-03-04,1791-03-03,NaN,NaN,NaN,NaN
4,C000538,George,Clymer,NaN,1739-03-16,M,NaN,PA,rep,1789-03-04,1791-03-03,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12755,H001104,Jon,Husted,Jon Husted,1967-08-25,M,Republican,OH,sen,2025-01-21,2029-01-03,https://www.husted.senate.gov,202-224-3353,304 Russell Senate Office Building Washington ...,304 Russell Senate Office Building
12756,M001244,Ashley,Moody,Ashley Moody,1975-03-28,F,Republican,FL,sen,2025-01-21,2029-01-03,https://www.moody.senate.gov,202-224-3041,387 Russell Senate Office Building Washington ...,387 Russell Senate Office Building
12757,P000622,Jimmy,Patronis,Jimmy Patronis,1972-04-13,M,Republican,FL,rep,2025-04-02,2027-01-03,https://patronis.house.gov,202-225-4136,2021 Rayburn House Office Building Washington ...,2021 Rayburn House Office Building
12758,F000484,Randall,Fine,Randy Fine,1974-04-20,M,Republican,FL,rep,2025-04-02,2027-01-03,https://fine.house.gov,202-225-2706,244 Cannon House Office Building Washington DC...,244 Cannon House Office Building


In [ ]:
index = out_df[out_df['Party'].isna()].index
index


for each in index:
    curr_name = out_df['Speaker'][each]
    print('Current Name: ', curr_name)
    print('Speech Date: ', speech_date[each])
    member_curr_index = []
    for i in members_df.index:
        if members_df['last_name'][i] == curr_name:
            member_curr_index.extend([i])
            #print('Member match ', members_df['last_name'][i])
            member_index = members_df
            speech_date = out_df['Date']
            for k in member_curr_index:
                start_date = pd.to_datetime(members_df['start'][k])
                end_date = pd.to_datetime(members_df['end'][k])
            print(members_df['last_name'][k], ' ', members_df['first_name'][k], ' ', members_df['start'][k], ' ', members_df['end'][k])
                    
    print('NEXTNEXTNEXTNEXT')

Current Name:  Courtney
Speech Date:  2019-06-13 00:00:00
Courtney   William   1947-01-03   1949-01-03
Courtney   Joe   2025-01-03   2027-01-03
NEXTNEXTNEXTNEXT
Current Name:  David P
Speech Date:  2019-10-15 00:00:00
NEXTNEXTNEXTNEXT
Current Name:  David P
Speech Date:  2019-10-15 00:00:00
NEXTNEXTNEXTNEXT
Current Name:  David P
Speech Date:  2019-10-15 00:00:00
NEXTNEXTNEXTNEXT
Current Name:  Hill
Speech Date:  2020-01-13 00:00:00
Hill   William   1801-12-07   1803-03-03
Hill   Mark   1821-12-03   1823-03-03
Hill   Isaac   1831-12-05   1837-03-03
Hill   John   1839-12-02   1841-03-03
Hill   John   1839-12-02   1841-03-03
Hill   Hugh   1847-12-06   1849-03-03
Hill   Clement   1853-12-05   1855-03-03
Hill   Ralph   1865-12-04   1867-03-03
Hill   Joshua   1871-03-04   1873-03-03
Hill   Benjamin   1877-10-15   1883-03-03
Hill   Nathaniel   1879-03-18   1885-03-03
Hill   John   1881-12-05   1883-03-03
Hill   William   1885-12-07   1887-03-03
Hill   Charles   1889-12-02   1891-03-03
Hill  

In [55]:
out_df

,Chamber,Session,Year,Date,Speaker,Party,State,Bill Reference,Title,URL,Volume,Number,Section,turn_number,speaker_title,speech_text,speech_length_words,granule_id,package_id
0,Extensions,165,2019,2019-06-13,Courtney,NaN,NaN,NaN,REINTRODUCING THE SERVICES FOR ENDING LONG-TER...,https://www.congress.gov/congressional-record/...,165,99,NaN,1,Mr.,"Madam Speaker, I rise today to congratulate No...",329,CREC-2019-06-13-pt1-PgE770,CREC-2019-06-13
1,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,1,Mr.,"Mr. Speaker, I move to suspend the rules and p...",234,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
2,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,2,Mr.,"Mr. Speaker, I ask unanimous consent that all ...",49,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
3,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,3,Mr.,"Mr. Speaker, I yield myself such time as I may...",500,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
4,House,165,2019,2019-10-15,David P,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,4,Mr.,"ROE of Tennessee. Mr. Speaker, I yield myself ...",321,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
5,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,5,Mr.,"Mr. Speaker, I yield 3 minutes to the gentlewo...",28,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
6,House,165,2019,2019-10-15,Brownley,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,6,Ms.,"Mr. Speaker, I thank the chairman for yielding...",435,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
7,House,165,2019,2019-10-15,Takano,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,7,Mr.,I yield the gentlewoman from California an add...,10,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
8,House,165,2019,2019-10-15,Brownley,Democrat,CA,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,8,Ms.,"Mr. Speaker, providing housing to our Nation's...",116,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
9,House,165,2019,2019-10-15,David P,NaN,NaN,NaN,HOMELESS VETERAN FAMILIES ACT; Congressional R...,https://www.congress.gov/congressional-record/...,165,162,NaN,9,Mr.,"ROE of Tennessee. Mr. Speaker, I yield 5 minut...",564,CREC-2019-10-15-pt1-PgH8135,CREC-2019-10-15
